This notebook creates a CSV file populated with Sephora product review data. 
The CSV is basically only the expanded "context_data_values" that is originally a JSON document converted to comma separated values. Columns with redundant information are removed.

The resulting CSV "reviews_mongo.csv" can then be used to import data to MongoDB

In [12]:
import pandas as pd,json

In [2]:
reviews_df = pd.read_csv('review_review.csv')

In [64]:
reviews_df = reviews_df.drop_duplicates('submission_id')

In [65]:
reviews_df.shape

(460599, 8)

In [62]:
reviews_df.head(5)

,submission_id,product_id,rating,review_text,submission_time,content_locale,is_recommended,context_data_values
0,190118856,P398965,5,Very moisturizing. When I apply it at night I ...,2021-11-06T03:16:05.000+00:00,en_CA,True,"{'skinType': {'Value': 'combination', 'Id': 's..."
1,188061530,P398965,4,I chose the sample as a reward in my previous ...,2021-09-29T19:52:05.000+00:00,en_US,False,"{'skinType': {'Value': 'oily', 'Id': 'skinType..."
2,186883060,P398965,5,"I got this as a sample, and I’ve been rationin...",2021-09-08T10:42:50.000+00:00,en_CA,True,"{'skinType': {'Value': 'combination', 'Id': 's..."
3,185227544,P398965,5,Lovveeee this lip conditioner. Been using for ...,2021-08-12T19:01:20.000+00:00,en_US,True,"{'skinType': {'Value': 'combination', 'Id': 's..."
4,184964050,P398965,5,THE BEST LIP CONDITIONER EVER. I use a differe...,2021-08-08T09:53:52.000+00:00,en_US,True,"{'skinType': {'Value': 'normal', 'Id': 'skinTy..."


In [50]:
eval([x for x in eval(reviews_df['context_data_values'].head(5).to_json()).values()][0])

{'skinType': {'Value': 'combination',
  'Id': 'skinType',
  'ValueLabel': 'Combination',
  'DimensionLabel': 'Skin Type'},
 'eyeColor': {'Value': 'blue',
  'Id': 'eyeColor',
  'ValueLabel': 'Blue',
  'DimensionLabel': 'Eye Color'},
 'StaffContext': {'Value': 'false',
  'Id': 'StaffContext',
  'ValueLabel': 'No',
  'DimensionLabel': 'I am a Sephora employee'},
 'hairColor': {'Value': 'blonde',
  'Id': 'hairColor',
  'ValueLabel': 'Blonde',
  'DimensionLabel': 'Hair color'},
 'skinTone': {'Value': 'light',
  'Id': 'skinTone',
  'ValueLabel': 'Light',
  'DimensionLabel': 'Skin Tone'},
 'IncentivizedReview': {'Value': 'false',
  'Id': 'IncentivizedReview',
  'ValueLabel': 'False',
  'DimensionLabel': 'I received this product as a free sample'}}

In [51]:
pd.json_normalize([eval(x) for x in eval(reviews_df['context_data_values'].head(5).to_json()).values()])

,skinType.Value,skinType.Id,skinType.ValueLabel,skinType.DimensionLabel,eyeColor.Value,eyeColor.Id,eyeColor.ValueLabel,eyeColor.DimensionLabel,StaffContext.Value,StaffContext.Id,...,hairColor.ValueLabel,hairColor.DimensionLabel,skinTone.Value,skinTone.Id,skinTone.ValueLabel,skinTone.DimensionLabel,IncentivizedReview.Value,IncentivizedReview.Id,IncentivizedReview.ValueLabel,IncentivizedReview.DimensionLabel
0,combination,skinType,Combination,Skin Type,blue,eyeColor,Blue,Eye Color,false,StaffContext,...,Blonde,Hair color,light,skinTone,Light,Skin Tone,false,IncentivizedReview,False,I received this product as a free sample
1,oily,skinType,Oily,Skin Type,brown,eyeColor,Brown,Eye Color,false,StaffContext,...,Black,Hair color,dark,skinTone,Dark,Skin Tone,false,IncentivizedReview,False,I received this product as a free sample
2,combination,skinType,Combination,Skin Type,brown,eyeColor,Brown,Eye Color,false,StaffContext,...,Brunette,Hair color,light,skinTone,Light,Skin Tone,true,IncentivizedReview,True,I received this product as a free sample
3,combination,skinType,Combination,Skin Type,brown,eyeColor,Brown,Eye Color,false,StaffContext,...,Black,Hair color,dark,skinTone,Dark,Skin Tone,false,IncentivizedReview,False,I received this product as a free sample
4,normal,skinType,Normal,Skin Type,blue,eyeColor,Blue,Eye Color,false,StaffContext,...,Blonde,Hair color,porcelain,skinTone,Porcelain,Skin Tone,false,IncentivizedReview,False,I received this product as a free sample


In [60]:
reviews_df.head(5)[['product_id', 'submission_id']]

,product_id,submission_id
0,P398965,190118856
1,P398965,188061530
2,P398965,186883060
3,P398965,185227544
4,P398965,184964050


In [70]:
pd.merge(reviews_df.head(5)[['product_id', 'submission_id']],pd.json_normalize([eval(x) for x in eval(reviews_df['context_data_values'].head(5).to_json()).values()]), left_index = True, right_index =True).to_csv('mongo_test.csv', index = False)

In [82]:
v = pd.json_normalize([eval(x) for x in eval(reviews_df['context_data_values'].to_json()).values()])
v = v[[x for x in v.columns if 'Value' in x and 'Label' not in x]]
v.columns = [x.replace(".Value", '') for x in v.columns]

In [72]:
k = reviews_df[['product_id', 'submission_id']]

In [91]:
mongo = pd.merge(k,v, left_index = True, right_index =True)

In [92]:
mongo.to_csv('reviews_mongo.csv', index = False)

In [76]:
mongo.shape

(460599, 50)

In [90]:
mongo

,product_id,submission_id,skinType,eyeColor,StaffContext,hairColor,skinTone,IncentivizedReview,age,VerifiedPurchaser,skinConcerns,beautyInsider,hairConcerns,hairCondition
0,P398965,190118856,combination,blue,false,blonde,light,false,NaN,NaN,NaN,NaN,NaN,NaN
1,P398965,188061530,oily,brown,false,black,dark,false,NaN,NaN,NaN,NaN,NaN,NaN
2,P398965,186883060,combination,brown,false,brunette,light,true,NaN,NaN,NaN,NaN,NaN,NaN
3,P398965,185227544,combination,brown,false,black,dark,false,NaN,NaN,NaN,NaN,NaN,NaN
4,P398965,184964050,normal,blue,false,blonde,porcelain,false,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
460594,2256337,190995944,oily,brown,false,black,olive,false,NaN,NaN,NaN,NaN,NaN,NaN
460595,2256345,190764703,combination,blue,false,auburn,light,false,NaN,True,NaN,NaN,NaN,NaN
460596,2243657,190894591,normal,brown,false,black,dark,false,NaN,False,NaN,NaN,NaN,NaN
460597,P432079,191193379,combination,brown,false,black,fair,false,NaN,NaN,acne,NaN,NaN,NaN


#### This notebook is used to create CSV file containing only review data and expanded JSON of reviewer attributes.
#### Resulting CSV is then imported to MongoDB.